In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import datetime
import statistics
import scipy.integrate as sp

In [ ]:
filenames = []
directory = "D:/Digital Lab/Lukas Auswertungen/Versuchsreihe_1/"
root_list = []
csv_list = []
for root,dirs,files in os.walk(directory):
    for file in files:
        if file.endswith(".csv"):
            root_list.append(root)
            csv_list.append(file)

In [ ]:
def plot_machine_data(column_list, converted_dates, versuch, leistung, power, filepath):
    for name, index in enumerate(column_list):
        fig, ax = plt.subplots()
        ax.plot_date(converted_dates, leistung[name], fmt='-', linewidth=0.15, markersize=0.05, 
                     color='black', label='Daten')
        ax.plot_date(converted_dates, [max(versuch[index]) for x in range(len(converted_dates))], 
                      color='red', fmt='-', linewidth=0.3, label='Max')
        ax.plot_date(converted_dates, [min(versuch[index]) for x in range(len(converted_dates))], 
                      color='yellow', fmt='-', linewidth=0.3, label='Min')
        ax.plot_date(converted_dates, [statistics.mean(versuch[index]) for x in range(len(converted_dates))], 
                      color='green', fmt='-', linewidth=0.3, label='Mean')
        plt.title('Achsleistung ' + str(index) +' - Energieverbrauch: ' + str(round(sum(power[name]), 3)) + ' Ws', fontsize=12)
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
        ax.legend(fancybox=True, loc='upper left' ,bbox_to_anchor=(1, 0.5))
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
        plt.ylabel(str(index) + ' [W]')
        plt.xlabel('Zeit')
        plt.savefig(filepath + ' ' + index + '.png', dpi=700)
        plt.close()
        #plt.savefig('./VERSUCH_01_UNBESCHICHTET/pythonplot_' + index + '.png', dpi=700)
        #plt.show()

In [ ]:
def evaluate_csv_monoblock(filepath):
    versuch = pd.read_csv(filepath, ';')
    columns = ['Timestamp', 'X1', 'Y1', 'SPI1']
    versuch = versuch[columns]
    # delete any row with 0's in SPI
    versuch = versuch[(versuch[['SPI1']] != 0).all(axis=1)]
    
    # remove median > 3 for all columns
    for key, value in versuch.items():
        if versuch[key].dtypes == 'float64':
            versuch.drop(versuch[value > (3 * statistics.mean(versuch[key]))].index, inplace=True)

    versuch.reset_index(inplace=True)
    # print(versuch.head())
    
    time_format = '%Y-%m-%dT%H:%M:%S,%f'
    power = []
    delta = []
    integral = []
    column_list = []
    leistung = []
    for key, value in versuch.iteritems():
        if key == 'Timestamp':
            for number, item in enumerate(value):
                if number < len(value)-1:
                    time_diff = datetime.datetime.strptime(value[number+1], time_format) - datetime.datetime.strptime(value[number], time_format)
                    delta.append(time_diff.total_seconds())
                else:
                    time_diff = datetime.datetime.strptime(value[number], time_format) - datetime.datetime.strptime(value[number-1], time_format)
                    delta.append(time_diff.total_seconds())


        if versuch[key].dtypes == 'float64':
            column_list.append(key)
            leistung.append(value)
            integral.append(abs(sp.cumtrapz(delta, value)))
            power.append(value * delta)
            #versuch.drop(versuch[value <= 0].index, inplace=True)
            
    date_list = []
    for item in versuch.Timestamp:
        date_list.append(datetime.datetime.strptime(item, time_format))

    converted_dates = matplotlib.dates.date2num(date_list)    
    plot_machine_data(column_list, converted_dates, versuch, leistung, power, filepath)

In [ ]:
filenames = []
for index, name in enumerate(root_list):
    csv_list[index] = csv_list[index].replace('._', '')
    path = str(root_list[index])+'/'+str(csv_list[index])
    filenames.append(path)
    filenames = (list(dict.fromkeys(filenames)))
    #print(filenames, index)
    #evaluate_csv_monoblock(filenames)
    
for length, path in enumerate(filenames):
    print(path)
    evaluate_csv_monoblock(path)
    
#[print(filenames[x]) for x in range(len(filenames))]